In [1]:
import pandas as pd
import matplotlib as plt
import numpy as np

In [2]:
# pd.np.set_printoptions(suppress=True)

In [3]:
pd.set_option('display.max_columns', None)

In [23]:
data = pd.read_csv('jan20_01-21_000000-235959.csv')

/usr/local/Cellar/jupyterlab/3.0.12/libexec/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (29) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [24]:
df_t = data[['rt_author_id', 't_author_id', 'rt_text', 't_text']]

In [25]:
df_t = df_t.dropna(subset=['t_text'])

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

In [7]:
cv = CountVectorizer(analyzer='word',token_pattern=r'[\w\@\#]+')

In [8]:
vec = cv.fit_transform(df_t.t_text.dropna().to_numpy())

In [9]:
labels = cv.get_feature_names() ## get all features/lables from the df

In [10]:
## get all mentions/hashtags and their indiceis
mentions = []
mention_indicies = []
hashtags = []
hashtag_indicies = []

In [11]:
## enumerate through them
for i, v in enumerate(labels):
    if v.startswith('@'):
        mentions.append(v)
        mention_indicies.append(i)
    elif v.startswith('#'):
        hashtags.append(v)
        hashtag_indicies.append(i)

In [12]:
## convert to np array to avoid sparse matrix
mentions = np.array(mentions)
mention_indicies = np.array(mention_indicies)
hashtags = np.array(hashtags)
hashtag_indicies = np.array(hashtag_indicies)

In [13]:
## get the columns vector for mentions and hashtags
mention_vec = vec[:,mention_indicies]
hashtag_vec = vec[:,hashtag_indicies]

In [14]:
## rank the indicies from all counts to get top mentions/tags
mentions_count = np.array(mention_vec.sum(axis=0)).flatten()
mentions_rank_indicies = mentions_count.argsort()[::-1]

hashtags_count = np.array(hashtag_vec.sum(axis=0)).flatten()
hashtags_rank_indicies = hashtags_count.argsort()[::-1]

In [15]:
## now get mention/hashtag ranked accordingly
mentions_ranked = mentions[mentions_rank_indicies]
hashtags_ranked = hashtags[hashtags_rank_indicies]

In [16]:
top10mentions_vec = vec[:,mention_indicies[mentions_rank_indicies[:10]]].toarray()
top10hashtags_vec = vec[:,hashtag_indicies[hashtags_rank_indicies[:10]]].toarray()

In [17]:
mentions_ranked.tolist()[::1][:20]

['@who',
 '@declanoscanlon',
 '@realdonaldtrump',
 '@bts_twt',
 '@drtedros',
 '@njsenatepres',
 '@doritmi',
 '@peterhotez',
 '@mcfunny',
 '@cnn',
 '@janeeopie',
 '@monstercoyliar',
 '@cdcgov',
 '@sheldon_walker_',
 '@thereal_truther',
 '@rosewind2007',
 '@nytimes',
 '@vingopal',
 '@beckyjohnson222',
 '@chrisjohnsonmd']

In [18]:
hashtags_ranked.tolist()[::1][:20]

['#coronavirus',
 '#china',
 '#wuhan',
 '#staysafe',
 '#2019ncov',
 '#vaccine',
 '#kashmir',
 '#wuhanpneumonia',
 '#breaking',
 '#vaccineswork',
 '#sars',
 '#vaccinesafety',
 '#hongkong',
 '#votenos2173',
 '#flu',
 '#virus',
 '#shenzhen',
 '#pneumonia',
 '#beijing',
 '#ปอดอ']

In [19]:
mention_indicies[mentions_rank_indicies[:-10]].shape

(46201,)

In [20]:
vec[:,mention_indicies[mentions_rank_indicies[:10]]].toarray().shape

(616837, 10)

In [26]:
df_t['mention_WHO'] = top10mentions_vec[:,0]
df_t['tag_CORONAVIRUS'] = top10hashtags_vec[:,0]

In [27]:
df_t[df_t.mention_WHO>0]

,rt_author_id,t_author_id,rt_text,t_text,mention_WHO,tag_CORONAVIRUS
3280,1105064943794245632,1.415915e+07,@UN @WHO Yup but Kashmir issues where are you?...,"As you start a new year, check out these 12 ti...",1,0
4881,2476070574,1.174222e+18,RT @TBG9270584: Maine Puts Vaccine Mandates to...,Maine Puts Vaccine Mandates to a Popular Vote ...,1,0
5490,1183658840077680640,1.174222e+18,RT @TBG9270584: Maine Puts Vaccine Mandates to...,Maine Puts Vaccine Mandates to a Popular Vote ...,1,0
6627,802504367245066240,4.476308e+07,RT @PATHtweets: With a recent approval from @W...,"With a recent approval from @WHO, a high-quali...",1,0
6841,1001089655758979072,2.090449e+07,"RT @jesswade: 🇨🇦💉Meet Prof Noni MacDonald, gam...","🇨🇦💉Meet Prof Noni MacDonald, gamechanging paed...",1,0
...,...,...,...,...,...,...
772187,2321485776,7.587032e+06,RT @SkyNews: The death toll from a measles epi...,The death toll from a measles epidemic in the ...,1,0
773518,414359347,9.311568e+07,RT @GaviSeth: Great list by @HelenBranswell. T...,Great list by @HelenBranswell. The Ervebo #Ebo...,1,0
774472,476068136,9.311568e+07,RT @GaviSeth: Great list by @HelenBranswell. T...,Great list by @HelenBranswell. The Ervebo #Ebo...,1,0
774711,1322115199,1.224144e+08,I will never forget that moment @HopeLizzy. Wh...,The @WHO designated 2020 as the #YearoftheNurs...,1,0


In [ ]:
# df_t[df_t.tag_CORONAVIRUS>0]

In [ ]:
# df_rt.to_csv('df_t_top_mntns_and_hashtags_jan_1-21.csv', index=False, sep=',')

In [ ]:
# data[data['rt_text'].astype(str).str.contains("@who|WHO")]
# df_rt[(df_rt['rt_text'].astype(str).str.contains("@who|WHO")) & (data['t_text'].astype(str).str.contains("@who|WHO"))][['rt_author_id', 't_author_id', 'rt_text', 't_text']]

In [ ]:
# data[(data['rt_text'].astype(str).str.contains("#coronavirus|coronavirus|corona|#corona")) & (data['t_text'].astype(str).str.contains("#coronavirus|coronavirus|corona|#corona"))][['rt_author_id', 't_author_id', 'rt_text', 't_text']]

In [ ]:
# get top 10 most frequent values
# data['t_author_id'].value_counts().index.tolist()[:10]

In [ ]:
# data.loc[np.logical_and(df.t_text == '@coronavirus', df.rt_text == '@who')]
# data.loc[df.rt_text =='@who'][['rt_author_id', 't_author_id', 'rt_text', 't_text']]